In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import cv2
import numpy
import pandas
#import imageio.v3 as io

#from tqdm import tqdm
from typing import Optional, Union
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
%matplotlib inline


In [3]:
# Comprobamos a abrirlos de nuevo /content/drive/MyDrive/datos/filtered_data_Test.csv
saved_data_Train = pandas.read_csv("/content/drive/MyDrive/datos/nuevo/filtered_data_Train.csv", sep=';')
saved_imgs_Train = numpy.load("/content/drive/MyDrive/datos/nuevo/imageTrain.npy")
saved_data_Train.shape, saved_imgs_Train.shape

y_train = saved_data_Train.iloc[:, 0:1]   # Seleccionamos la primera columna, price
X_train = saved_data_Train.iloc[:, 1:-1]  # Seleccionamos todas las columnas excepto la primera y la última

scaler = preprocessing.StandardScaler().fit(X_train)
XtrainScaled = scaler.transform(X_train)

X_train = XtrainScaled

In [4]:
# Comprobamos a abrirlos de nuevo
saved_data_Test = pandas.read_csv("/content/drive/MyDrive/datos/nuevo/filtered_data_Test.csv", sep=';')
saved_imgs_Test = numpy.load("/content/drive/MyDrive/datos/nuevo/imagesTest.npy")
saved_data_Test.shape, saved_imgs_Test.shape

y_test = saved_data_Test.iloc[:, 0:1]      # nos quedamos con la 1ª columna, price
X_test = saved_data_Test.iloc[:,1:-1]      # nos quedamos con el resto


# recordad que esta normalización/escalado la realizo con el scaler anterior, basado en los datos de training!
XtestScaled = scaler.transform(X_test)
X_test = XtestScaled

In [5]:
# Comprobamos a abrirlos de nuevo
saved_data_Val = pandas.read_csv("/content/drive/MyDrive/datos/nuevo/filtered_data_Val.csv", sep=';')
saved_imgs_Val = numpy.load("/content/drive/MyDrive/datos/nuevo/imagesVal.npy")
saved_data_Val.shape, saved_imgs_Val.shape

y_val = saved_data_Val.iloc[:,0:1]     # nos quedamos con la 1ª columna, price
X_val = saved_data_Val.iloc[:,1:-1]      # nos quedamos con el resto


# recordad que esta normalización/escalado la realizo con el scaler anterior, basado en los datos de training!
XvalScaled = scaler.transform(X_val)

X_val = XvalScaled

In [6]:
# Cargar los datos
X_train_img = saved_imgs_Train
X_test_img = saved_imgs_Test
X_val_img = saved_imgs_Val

# Late-fusion
La idea ha sigo pasar los datos a travez de arqutecturas sencillas con el fin de unir la salida y entonces aplicar una capa densa que devuelva la predicion del precio. En este caso se usa redes sencillas con el fin de agilizar el training del conjuto.

In [8]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, concatenate
from tensorflow.keras.optimizers import Adam

# Definir la arquitectura de la red neuronal para las imágenes
img_input = Input(shape=(224, 224, 3))
x = Flatten()(img_input)
x = Dense(128, activation='relu')(x)
img_output = Dense(64, activation='relu')(x)

# Definir la arquitectura de la red neuronal para los datos tabulares
tab_input = Input(shape=(X_train.shape[1],))
x = Dense(64, activation='relu')(tab_input)
tab_output = Dense(32, activation='relu')(x)

# Combinar las salidas de ambas redes( se fusionan las salidas de las 
# redes neuronales convolucionales y las redes neuronales densas después de entrenarlas por separado)
combined = concatenate([img_output, tab_output])
x = Dense(32, activation='relu')(combined)
output = Dense(1, activation='linear')(x)

# Crear el modelo final
model = Model(inputs=[img_input, tab_input], outputs=output)

# Compilar el modelo
learning_rate = 0.001
opt = Adam(learning_rate=learning_rate)
model.compile(loss='mean_squared_error', optimizer=opt)

# Entrenar el modelo
history = model.fit(
    [X_train_img, X_train], y_train,
    validation_data=([X_val_img, X_val], y_val),
    epochs=10, batch_size=16,
)

# Evaluar el modelo en los datos de prueba
mse = model.evaluate([X_test_img, X_test], y_test)
print("Mean Squared Error:", mse)

Epoch 1/10
423/423 [==============================] - 177s 414ms/step - loss: 1991.0874 - val_loss: 1846.2032
Epoch 2/10
423/423 [==============================] - 227s 535ms/step - loss: 1439.2205 - val_loss: 1888.2312
Epoch 3/10
423/423 [==============================] - 195s 460ms/step - loss: 1356.5546 - val_loss: 1663.4097
Epoch 4/10
423/423 [==============================] - 183s 433ms/step - loss: 1295.4856 - val_loss: 1637.1975
Epoch 5/10
423/423 [==============================] - 187s 441ms/step - loss: 1255.9296 - val_loss: 1612.7627
Epoch 6/10
423/423 [==============================] - 182s 431ms/step - loss: 1222.2620 - val_loss: 1571.6996
Epoch 7/10
423/423 [==============================] - 169s 400ms/step - loss: 1191.5258 - val_loss: 1596.1163
Epoch 8/10
423/423 [==============================] - 178s 422ms/step - loss: 1187.5475 - val_loss: 1539.9985
Epoch 9/10
423/423 [==============================] - 180s 425ms/step - loss: 1165.6250 - val_loss: 1529.3176
Epoch 10/1

Podemos ver en el transcurso de las epocas que se esta aprendiendo, ya que en train y validacion.

# Early-fusion
Lo que haremos es la union de los datos, asi que los datos tabulares y las imagenes se aplanaran en un solo vextor de entrada y acontinuacion pasaran por dos capas densas con el fin de agilizar el training.

In [7]:
import numpy as np
from keras.models import Model
from keras.layers import concatenate, Input, Flatten, Dense
from keras.optimizers import Adam
from keras.applications import VGG16

In [ ]:


# Definir la arquitectura de la red de imágenes preentrenada
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
image_input = Input(shape=(224, 224, 3))
image_features = base_model(image_input)
image_features = Flatten()(image_features)

# Definir la arquitectura de la red para los datos tabulares
tabular_input = Input(shape=(X_train.shape[1],))
tabular_features = Dense(64, activation='relu')(tabular_input)

# Fusionar los datos de entrada de imágenes y tabulares (En este punto tenemos un vector 
# procedente de la estracion de caracteristivas de VGG que se use al vector de datos tabulares )
combined_input = concatenate([image_features, tabular_features])

# Definir la arquitectura de la red neuronal. Aqui agregamos las capas densas para obtener la salida
x = Dense(128, activation='relu')(combined_input)
x = Dense(64, activation='relu')(x)
output = Dense(1, activation='linear')(x)

# Crear el modelo final
model = Model(inputs=[image_input, tabular_input], outputs=output)

# Compilar el modelo
learning_rate = 0.001
opt = Adam(learning_rate=learning_rate)
model.compile(loss='mean_squared_error', optimizer=opt)

# Entrenar el modelo
history = model.fit(
    [X_train_img, X_train], y_train,
    validation_data=([X_val_img, X_val], y_val),
    epochs=10, batch_size=16,
)

# Evaluar el modelo en los datos de prueba
mse = model.evaluate([X_test_img, X_test], y_test)
print("Mean Squared Error:", mse)


la idea en este cuaderno es mostrar el conocimiento de los terminos aunque sea en una arquitectura simple. No se busca obtener los mejores resultados sino una configuracion acorde a los conceptos pedidos.